In [ ]:
!wget https://dataset.cs.mcgill.ca/ubuntu-corpus-1.0/ubuntu_dataset.tgz.aa
!wget https://dataset.cs.mcgill.ca/ubuntu-corpus-1.0/ubuntu_dataset.tgz.ab
!wget https://dataset.cs.mcgill.ca/ubuntu-corpus-1.0/ubuntu_dataset.tgz.ac
!wget https://dataset.cs.mcgill.ca/ubuntu-corpus-1.0/ubuntu_dataset.tgz.ad
!wget https://dataset.cs.mcgill.ca/ubuntu-corpus-1.0/ubuntu_dataset.tgz.ae

--2024-02-05 15:24:24--  https://dataset.cs.mcgill.ca/ubuntu-corpus-1.0/ubuntu_dataset.tgz.aa
Resolving dataset.cs.mcgill.ca (dataset.cs.mcgill.ca)... 132.206.51.25
Connecting to dataset.cs.mcgill.ca (dataset.cs.mcgill.ca)|132.206.51.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1000000000 (954M) [application/octet-stream]
Saving to: ‘ubuntu_dataset.tgz.aa.1’

ubuntu_dataset.tgz.  48%[========>           ] 463.01M  10.7MB/s    eta 31s    

## load dataset(almost a failure :) )

In [ ]:
!cat ubuntu_dataset.tgz.a* | tar xz

In [ ]:
# !tar -xvf ubuntu_dataset.tgz.aa

In [ ]:
# import pandas as pd
# import gc
# # Specify the chunk size
# chunk_size = 10**8  # Adjust the chunk size based on your available memory

# # Initialize an empty DataFrame to store the concatenated data
# concatenated_df = pd.DataFrame()

# # Iterate over chunks and process them
# for chunk in pd.read_csv('/content/ubuntu_csvfiles/trainset.csv', chunksize=chunk_size):
#     # Process each chunk (perform any required operations or filtering)
#     # For example, you might want to filter rows or perform computations on the chunk
#     processed_chunk = chunk  # Placeholder, replace with your actual processing logic
#     gc.collect()
#     concatenated_df = pd.concat([concatenated_df, processed_chunk], ignore_index=True)
#---------------------------------------------
# from dask.distributed import Client, LocalCluster
# import dask.dataframe as dd
# import pandas as pd
# cluster = LocalCluster()
# client = Client(cluster)
# chunk_size = 10**8

# dask_df = dd.read_csv('/content/ubuntu_csvfiles/trainset.csv', blocksize=chunk_size)


# # Visualize the Dask DataFrame partitions
# # dask_df.visualize()

# # Perform operations on Dask DataFrame


# # Convert Dask DataFrame to Pandas DataFrame if needed
# result_pandas =dask_df.compute()
# client.close()
# cluster.close()
#-------------------------------# import dask.dataframe as dd
# import pandas as pd
# chunk_size = 40000  # Adjust the chunk size based on your available memory

# # Read the CSV file using Dask and specify the number of partitions
# # dask_df = dd.read_csv('/content/ubuntu_csvfiles/trainset.csv', blocksize=chunk_size)


# # Visualize the Dask DataFrame partitions
# # dask_df.visualize()

# # Perform operations on Dask DataFrame


# # Convert Dask DataFrame to Pandas DataFrame if needed
# result_pandas =dask_df.compute()

In [ ]:
# !tar -xvf ubuntu_blobs.tgz

## import 0.25 of data :)

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Read the entire dataset from the CSV file
full_dataset = pd.read_csv('/content/ubuntu_csvfiles/trainset.csv')

# Define the desired subset size (adjust as needed)


# Save the subset to a new CSV file



In [ ]:
subset_size = int(0.03 * len(full_dataset))  # 25% of the full dataset

# Randomly sample a subset from the full dataset
mydataset = full_dataset.sample(n=subset_size, random_state=42)  # Set a random seed for reproducibility

In [ ]:
print(len(mydataset))

## Tokenize and preprocess data

In [ ]:
mydataset

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, get_linear_schedule_with_warmup
import pandas as pd

# Load the model and tokenizer
model_name = "microsoft/DialoGPT-medium"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a padding token

model = AutoModelForCausalLM.from_pretrained(model_name)
print(model.config.max_position_embeddings)
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# import torch
# import torch_xla
# import torch_xla.core.xla_model as xm

# device = xm.xla_device()
# Load your dataset (assuming 'context' and 'utterance' are column names)
# Adjust this part based on your actual dataset structure
# Replace 'context' and 'utterance' with the actual column names
# mydataset = pd.read_csv('/content/ubuntu_csvfiles/trainset.csv')
# mydataset = mydataset[['context', 'utterance']]

# Define your custom dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=32):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data.iloc[idx][0]
        response = self.data.iloc[idx][1]

        # Truncate or omit tokens if the total length is greater than max_length
        input_text = f"{context} </s> {response}"
        encoding = self.tokenizer(
            input_text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=self.max_length,
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': encoding['input_ids'].flatten()
        }


custom_dataset = MyDataset(mydataset, tokenizer)
# Create a DataLoader
data_loader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

# Set up training parameters
epochs = 2
learning_rate = 2e-5

# Create optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(data_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)



In [ ]:
# Training loop
model.resize_token_embeddings(len(tokenizer))
model.to(device)

# export CUDA_LAUNCH_BLOCKING=1
# import os

# # Set CUDA_LAUNCH_BLOCKING environment variable
# os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

In [ ]:
# Set CUDA_LAUNCH_BLOCKING environment variable
import os

# model.resize_token_embeddings(len(tokenizer))
for epoch in range(epochs):
    i = 1
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Ensure the length of input_ids is within the model's limit
        # if input_ids.size(1) > model.config.max_position_embeddings:
        #     input_ids = input_ids[:, :model.config.max_position_embeddings]
        # print(input_ids.shape)

        outputs = model(input_ids, labels=labels)

        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        if(i %1000 == 0):
          print("1000 batch is passed")
        i+=1
    print("batch f{batch}")

# Save the pretrained model
model.save_pretrained("pretrained_dialogpt_model")
